# SVD Golub-Reinsch algorithm (written in Julia)

#### Author: João Vitor de Oliveira Silva

In [1]:
using LinearAlgebra
using PlotlyJS

WebIO._IJuliaInit()

In [2]:
function householder(x)
    """Computes the Householder transformation for input vector x
    
    Returns 
    beta: float, the multiplier for future Householder reflection
    v: vector, the householder reflector vector  
    """
    sigma = dot(x[2:end],x[2:end])
    v = copy(x)

    if sigma == 0
        beta = 0
        return beta, v
    end

    sq = sqrt(x[1]^2 + sigma)
    if x[1] > 0
        v[1] += sq
    else
        v[1] -= sq
    end

    beta = 2.0 / (v[1]^2 + sigma)

    return beta, v
end


function apply_householder_col!(B, H, cidx, m)
    """Apply Householder reflection from the left (B<- H * B)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[cidx:m, cidx])
    B[cidx:m, cidx:end] = B[cidx:m, cidx:end] - beta * v * (v' * B[cidx:m, cidx:end])
    H[cidx:end, cidx] = v / norm(v)

    return beta
end

function apply_householder_row!(B, H, ridx, n)
    """Apply Householder reflection from the right (B<- B * H)
    
    Returns (inplace) 
    B: input matrix
    H: householder vectors v  
    """
    beta, v = householder(B[ridx, ridx+1:n])
    B[ridx:end, ridx+1:n] = B[ridx:end, ridx+1:n] - (B[ridx:end, ridx+1:n] * v) * (beta * v')
    H[ridx, ridx+1:end] = v / norm(v)

end

function update_col_sim!(U, H, cidx, m)
    """Update column unitary transform (U<- H * U)
    
    Input:
    U: Left Householder matrix (orthogonal)
    H: householder vectors v
    cidx: index of column where householder transform is applied
    m: number of columns of original matrix
    
    Returns (inplace) 
    U: Left Householder matrix (orthogonal)
    """
    v = H[cidx:end, cidx]
    if cidx > 1
        v = [zeros(cidx-1);v]
    end
    
    
    
    U[:,:] = U - 2 * v * (v' * U)
    
    return U
    
end

function update_row_sim!(V, H, ridx, n)
    """Update row unitary transform (V<- V * H)
    
    Input:
    V: Right Householder matrix (orthogonal)
    H: householder vectors v
    ridx: index of row where householder transform is applied
    n: number of rows of original matrix
    
    Returns (inplace) 
    V: Right Householder matrix (orthogonal) 
    """
    v = H[ridx, ridx+1:end]
    v = [zeros(ridx);v]
    
    

    V[:,:] = V - 2 * (V * v) * v'
    
    return V
    
end


function bidiagonalize(A, return_orth = false)
    """Performs matrix bidiagonalization (for future singular value decomposition computation)
    
    Input:
    A: Rectangular m x n matrix
    
    Returns (inplace) 
    B: bidiagonal matrix with same singular values as input matrix (A = U * B * V')
    """
    m, n = size(A)
    B = copy(A)
    H = copy(A)  # v vectors in householder
    
    if return_orth
        U = zeros(m, m) + I
        V = zeros(n, n) + I
    end
    
    for k = 1:min(m,n)
        
        # column
        
        apply_householder_col!(B, H, k, m)
    
        
        if return_orth
            update_col_sim!(U, H, k, m)
        end
        
        
        # row
        if k < n
            apply_householder_row!(B, H, k, n) 
            
            if return_orth 
                update_row_sim!(V, H, k, n)
            end
        end
        
                
    end
    
    if return_orth
        V[:, end] = -V[:, end] 
        U[end, :] = -U[end, :]
        return U', B, V
    else
        return B, H
    end
end



bidiagonalize (generic function with 2 methods)

In [3]:
function givens_rotation(v)
    """ Computers the 2x2 Givens Rotation matrix for a given 2d vector v.
    
    Input:
    v: 2d vector
    
    Returns
    G:  givens rotation matrix
    """
    a, b = v[1], v[2]
    if b == 0
        c = 1
        s = 0
    else
        if abs(b) > abs(a)
            tau = -a/b
            s = 1.0/sqrt(1.0+tau*tau)
            c = s*tau
        else
            tau = -b/a
            c = 1.0/sqrt(1.0+tau*tau)
            s = c*tau
        end
    end
    return [c -s;s c]
end


function svd_golub_reinsh!(B, U, V, maxiter, eps=1e-12)
    """Computes the Singular value decomposition of a bidiagonal matrix. Based on 
    https://www.cs.utexas.edu/~inderjit/public_papers/HLA_SVD.pdf
    and
    https://link.springer.com/article/10.1007/s11075-022-01459-9
    
    Returns  
    U: Left singular vector matrix
    S: Singular values matrix
    Vt: Right singular vector matrix
    """
    
    function golub_kahan_step!(B, U, V, l, k)
        # Considering B22 -> (l:k x l:k)
        
        # Step 1: compute appropriate shift
        
        if k > 2
            a, b, d = B[k-1, k-1]^2 + B[k-2, k-1]^2, B[k-1, k-1] * B[k-1, k], B[k, k]^2 + B[k-1, k]^2
            t, r = - a - d, a * d - b^2
            s_1, s_2 = 0.5 * (-t + sqrt(t^2 -4 * r)), 0.5 * (-t - sqrt(t^2 -4 * r))
            if abs(B[k, k]^2 + B[k-1, k]^2 - s_1) < abs(B[k, k]^2 + B[k-1, k]^2 - s_2)
                s = s_1 
            else
                s = s_2 
            end
        else
           s = B[2, 2]^2 + B[1, 2]^2 
        end
        
        # Step 2: Computing Givens Rotations
        alpha, beta = B[l, l]^2-s, B[l, l] * B[l, l + 1]
        v = [alpha;beta]
        for p = l:k-1
            # Right rotation
            G = givens_rotation(v)
            B[:, p:p+1] = B[:, p:p+1] * G'
            V[:, p:p+1] = V[:, p:p+1] * G'
            
            
            # Left rotation
            alpha, beta = B[p, p], B[p + 1, p] 
            v = [alpha; beta]
            G = givens_rotation(v)
            B[p:p+1, :] = G * B[p:p+1, :]
            U[:, p:p+1] = U[:, p:p+1] * G'
            
            # Updating alpha, beta and v
            if p < k -1 
                alpha, beta = B[p, p+1], B[p, p+2]
                v = [alpha;beta]
            end
        end
    end
    
    m, n = size(B)
    maxiter_ = log2(n * m) * 30 
    
    for iters = 1:min(maxiter, maxiter_)
        
        # Step 1 of main loop
        # Find l and k such that matrix can be blocked as B11, B22 and B33 (diag) 
        
        k = n
        for j = n:-1:2  
            k = abs(B[j-1, j]) < eps * (abs(B[j, j]) + abs(B[j-1, j-1]))  ? j-1 : k
        end

        
        if k == 1
           # Diagonal matrix, convergence obtained
           return U, B, V
        end
        
        # Step 2 of main loop
        # Performing Golub Kahan at matrix B22
        if k == n 
            # B22 should be the full matrix 
            golub_kahan_step!(B, U, V, 1, n)
    
        else 
            # B33 (diagonal) found, have to check where B22 starts (l) and ends (k)
            l = k
            for j = k:-1:2
                l = abs(B[j-1, j]) < eps * (abs(B[j, j]) + abs(B[j-1, j-1])) ? j-1 : l
                
                if l!=k
                    break
                end
                
            end 
            
            # If l is equal to k after previous scan, then l = 1 and there is no B11 matrix
            l = (l == k) ? 1 : l 
            
            golub_kahan_step!(B, U, V, l, k)
        end
    end
end


function svd_num(A, maxiter = 80, save_results = false)
   """Performs singular value decompositon of a rectangular m x n matrix A.
    
    Returns 
        U: Left singular vector matrix
        S: Singular values matrix
        Vt: Right singular vector matrix
    """ 
    # Flag for transpose (case n > m)
    m, n = size(A)
    transpose_flag = false
    
    if n > m
        transpose_flag = true
    end
    
    # First main step: Find bidiagonalized matrix B
    if transpose_flag
        P_l, B, P_r = bidiagonalize(A', true);
    else
        P_l, B, P_r = bidiagonalize(A, true);
    end
    
    # Second main step: perform svd_golub_reinsh algorithm iterations
    if save_results
        t = min(m, n)
        results = zeros(2, t-1, maxiter)
        
        for k = 1:maxiter
            for j = 2:t
                results[:, j-1, k] = copy(B[j-1:j, j])
            end
            svd_golub_reinsh!(B, P_l, P_r, 1)
        end
        
        B = [i == j ? B[i, j] : 0 for i in 1:size(B, 1), j in 1:size(B, 2)];
        # Check if transpose
        if transpose_flag
            return P_r, B', P_l, results
        else
            return P_l, B, P_r, results
        end
        
    else
        svd_golub_reinsh!(B, P_l, P_r, maxiter)
        B = [i == j ? B[i, j] : 0 for i in 1:size(B, 1), j in 1:size(B, 2)];
        
        # Check if transpose
        if transpose_flag
            return P_r, B', P_l
        else
            return P_l, B, P_r
        end
    end
    
    
    
    
    
end

svd_num (generic function with 3 methods)

## Initial tests

In [4]:
# Example usage 1:
A = randn(5, 7)

5×7 Matrix{Float64}:
 -2.12368   -0.412161  -0.0147965  …  -1.15317     1.26409      0.342208
  0.759364   0.453807  -0.166023       0.140065    0.00505375   0.0637178
 -1.68399   -0.698247   1.09471       -1.10204    -0.474134    -0.237761
 -1.3459     0.519392   2.03642        0.834496   -0.81838     -0.867635
 -0.664309  -1.4317    -1.1144        -0.0672647  -1.13022     -0.523552

In [5]:
A

5×7 Matrix{Float64}:
 -2.12368   -0.412161  -0.0147965  …  -1.15317     1.26409      0.342208
  0.759364   0.453807  -0.166023       0.140065    0.00505375   0.0637178
 -1.68399   -0.698247   1.09471       -1.10204    -0.474134    -0.237761
 -1.3459     0.519392   2.03642        0.834496   -0.81838     -0.867635
 -0.664309  -1.4317    -1.1144        -0.0672647  -1.13022     -0.523552

In [6]:
P_l, B, P_r = bidiagonalize(A, true);

In [7]:
B

5×7 Matrix{Float64}:
  3.18986      -1.91977      …  -2.22045e-16  0.0      -5.55112e-17
  0.0          -0.918846         0.0          0.0       0.0
  0.0          -1.11022e-16      0.0          0.0       0.0
 -2.22045e-16  -3.46945e-18     -2.36681      0.0       0.0
  0.0           0.0              0.04101      0.58381  -5.55112e-17

In [8]:
show(stdout, "text/plain", opnorm(P_l * B * P_r' - A, 1))

1.9984014443252818e-15

In [9]:
U, S, V = svd_num(A);

In [10]:
show(stdout, "text/plain", U)

5×5 Matrix{Float64}:
  0.592177   0.548588     0.381771   -0.413801   0.177192
 -0.204677  -0.0237641    0.157842    0.223514   0.939505
  0.625909   0.00274185  -0.130451    0.768513  -0.0244896
  0.449744  -0.799568    -0.0173468  -0.361074   0.166571
  0.115764   0.243249    -0.901121   -0.240464   0.239974

In [11]:
show(stdout, "text/plain", S)

5×7 adjoint(::Matrix{Real}) with eltype Real:
 3.75907   0        0        0          0         0  0
 0        -2.90981  0        0          0         0  0
 0         0        2.38342  0          0         0  0
 0         0        0        0.902042   0         0  0
 0         0        0        0         -0.583758  0  0

In [12]:
show(stdout, "text/plain", V')

7×7 Matrix{Float64}:
 -0.837776    -0.18785    0.398309   0.128446  -0.275016   -0.0128043  -0.109078
  0.09387      0.344474   0.653136  -0.192305   0.454519   -0.368227   -0.258417
  0.0632485    0.539767   0.333227   0.264203  -0.0957615   0.662036    0.276306
  0.443491    -0.119611   0.380238   0.144702  -0.691304   -0.353707    0.143108
  0.00896872  -0.194205   0.194648  -0.874141  -0.132091    0.286412    0.246402
  0.278183    -0.681728   0.331463   0.252563   0.284671    0.394095   -0.218762
  0.105685     0.194747  -0.10671   -0.166935  -0.363623    0.251995   -0.846124

In [13]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

3.4710590843684775e-14

In [14]:
show(stdout, "text/plain", opnorm(U * U' - I, 1))

1.7092604894345183e-15

In [15]:
show(stdout, "text/plain", opnorm(V * V' - I, 1))

2.759197333006883e-15

In [16]:
# Example usage 2:
A = randn(4, 3)

4×3 Matrix{Float64}:
 -1.26641   -2.43672   0.965088
  0.433606   1.68572   1.33091
  0.495973   0.246087  0.865366
  0.468738   0.770922  0.374275

In [17]:
U, S, V = svd_num(A);

In [18]:
show(stdout, "text/plain", U)

4×4 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.79741   -0.104564  -0.57398   -0.154113
  0.521819  -0.504993  -0.671978   0.145369
  0.138746   0.820707  -0.434615   0.343946
  0.26943    0.245943  -0.173497  -0.914778

In [19]:
show(stdout, "text/plain", S)

4×3 Matrix{Real}:
 -3.38543   0          0
  0        -0.482444   0
  0         0         -1.89457
  0         0          0

In [20]:
show(stdout, "text/plain", V')

3×3 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.422758   -0.905218   -0.0430753
 -0.903284    0.424739   -0.0606269
 -0.0731764  -0.0132787   0.997231

In [21]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

1.4203319768857191e-15

In [22]:
show(stdout, "text/plain", opnorm(U * U' - I, 1))

1.8504743799811473e-15

In [23]:
show(stdout, "text/plain", opnorm(V * V' - I, 1))

1.75619067899202e-15

In [24]:
# Example usage 3:
A = randn(7, 7)
A = A + A'

7×7 Matrix{Float64}:
 -0.123759  -0.482065  -0.376651   2.25838   -1.94773   -1.92126    0.13943
 -0.482065   0.860089  -1.03324   -0.692005  -0.219111   0.71684    2.31767
 -0.376651  -1.03324    1.74975    0.358937   1.51249    3.85002    0.58522
  2.25838   -0.692005   0.358937   1.17458    0.938756   2.0232    -0.31107
 -1.94773   -0.219111   1.51249    0.938756   0.323364   0.269959  -0.178144
 -1.92126    0.71684    3.85002    2.0232     0.269959  -1.50794    2.65572
  0.13943    2.31767    0.58522   -0.31107   -0.178144   2.65572    0.303298

In [25]:
U, S, V = svd_num(A);

In [26]:
show(stdout, "text/plain", U)

7×7 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.394307    0.225875    0.286002   -0.198491   -0.566819  -0.587343    0.0777879
  0.0332425  -0.0737868  -0.622573    0.320824   -0.284279  -0.266407   -0.592569
  0.331821   -0.642004    0.228417   -0.437603    0.133827  -0.375927   -0.273542
  0.337208   -0.192746    0.504727    0.458886   -0.534868   0.291359   -0.11331
 -0.203235   -0.285229    0.133888    0.666713    0.36865   -0.453205    0.27131
 -0.712587   -0.556768   -0.0483545  -0.0782458  -0.13489    0.387828   -0.0718543
  0.265553   -0.320733   -0.451019   -0.0705068  -0.373911  -0.0451575   0.690199

In [27]:
show(stdout, "text/plain", S)

7×7 Matrix{Real}:
 -6.26739   0         0        0        0        0         0
  0        -6.17448   0        0        0        0         0
  0         0        -3.80346  0        0        0         0
  0         0         0        0.43835  0        0         0
  0         0         0        0        2.75601  0         0
  0         0         0        0        0        1.92734   0
  0         0         0        0        0        0        -2.19819

In [28]:
show(stdout, "text/plain", V')

7×7 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.394307    0.0332425   0.331821   0.337208  -0.203235  -0.712587    0.265553
 -0.225875    0.0737868   0.642004   0.192746   0.285229   0.556768    0.320733
 -0.286002    0.622573   -0.228417  -0.504727  -0.133888   0.0483545   0.451019
 -0.198491    0.320824   -0.437603   0.458886   0.666713  -0.0782458  -0.0705068
 -0.566819   -0.284279    0.133827  -0.534868   0.36865   -0.13489    -0.373911
  0.587343    0.266407    0.375927  -0.291359   0.453205  -0.387828    0.0451575
  0.0777879  -0.592569   -0.273542  -0.11331    0.27131   -0.0718543   0.690199

In [29]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

4.758158165285885e-13

In [30]:
show(stdout, "text/plain", opnorm(U * U' - I, 1))

2.3511560116144236e-15

In [31]:
show(stdout, "text/plain", opnorm(V * V' - I, 1))

2.7310052653166084e-15

## Cubic convergence depiction

### Square 7 x 7

In [32]:
A = randn(7, 7)

7×7 Matrix{Float64}:
 -0.0591793  -1.20341     1.39422    …   1.23926    0.268111   2.48939
  1.17793     0.0739997  -3.27643       -0.451675  -0.453596   1.72729
 -1.37841    -0.0118133  -0.0322053      3.1592     1.68904    0.574639
  0.69807     2.10219     1.49995       -1.07232    0.4721    -1.77117
 -0.556374   -0.100701    3.1333         1.60847    0.442456  -1.51995
 -0.377764   -0.140365   -2.62814    …   1.48052   -0.137452   2.39138
 -1.04846    -0.772259   -1.315          0.222495  -0.742677   1.41022

In [33]:
U, S, V, results = svd_num(A, 20, true);

In [34]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

9.154846696546655e-15

In [35]:
function plot_convergence_error(error_vectors, labels)
    iterations = 1:length(error_vectors[1])
    traces = Vector{GenericTrace{Dict{Symbol, Any}}}()
    
    for (i, error_vector) in enumerate(error_vectors)
        trace = scatter(
            x = iterations,
            y = error_vector,
            mode = "lines+markers",
            marker_color = i,
            line_shape = "linear",
            name = labels[i]
        )
        push!(traces, trace)
    end
    
    layout = Layout(
        title = "Value of element above diagonal σ_i",
        xaxis_title = "Iteration",
        yaxis_title = "Error",
        yaxis_type = "log",
        yaxis_tickformat = "e",
        showlegend = true
    )
    
    plot(traces, layout)
end

error_vectors = [
    abs.(results[1, 1, :]),
    abs.(results[1, 2, :]),
    abs.(results[1, 3, :]),
    abs.(results[1, 4, :]),
    abs.(results[1, 5, :]),
    abs.(results[1, 6, :])
]
labels = ["σ2", "σ3", "σ4", "σ5", "σ6", "σ7"]
plot_convergence_error(error_vectors, labels)

data: [
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, showlegend, template, title, xaxis, and yaxis"

In [36]:
# σ_2
show(stdout, "text/plain", results[2, 1, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -3.27529  -2.19199  6.02272  6.9265  6.89722  6.76767  6.47813  6.01052  5.6201  5.40634  5.32748  5.327  5.327  5.327  5.327  5.327  5.327  5.327  5.327  5.327

In [37]:
# σ_3
show(stdout, "text/plain", results[2, 2, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -1.99341  6.08466  -2.74072  -2.90762  -3.05681  -3.07645  -3.07866  -3.07887  -3.07887  -3.07887  -3.07887  -3.07887  -3.07887  -3.07887  -3.07887  -3.07887  -3.07887  -3.07887  -3.07887  -3.07887

In [38]:
# σ_4
show(stdout, "text/plain", results[2, 3, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 4.96941  -2.83157  -2.43876  -1.99657  -1.89382  -1.88157  -1.88021  -1.88008  -1.88008  -1.88008  -1.88008  -1.88008  -1.88008  -1.88008  -1.88008  -1.88008  -1.88008  -1.88008  -1.88008  -1.88008

In [39]:
# σ_5
show(stdout, "text/plain", results[2, 4, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -2.6785  -1.09726  -1.02011  -1.02048  -1.02082  -1.02082  -1.02082  -1.02082  -1.02082  -1.02082  -1.02082  -1.02082  -1.02082  -1.02082  -1.02082  -1.02082  -1.02082  -1.02082  -1.02082  -1.02082

In [40]:
# σ_6
show(stdout, "text/plain", results[2, 5, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -0.866558  -0.815708  -0.805577  -0.803878  -0.803554  -0.803554  -0.803554  -0.803554  -0.803554  -0.803554  -0.803554  -0.803554  -0.803554  -0.803554  -0.803554  -0.803554  -0.803554  -0.803554  -0.803554  -0.803554

In [41]:
# σ_7
show(stdout, "text/plain", results[2, 6, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -0.627539  -0.618636  -0.618626  -0.618626  -0.618626  -0.618626  -0.618626  -0.618626  -0.618626  -0.618626  -0.618626  -0.618626  -0.618626  -0.618626  -0.618626  -0.618626  -0.618626  -0.618626  -0.618626  -0.618626

In [42]:
S

7×7 Matrix{Real}:
 6.97259  0       0         0         0         0          0
 0        5.327   0         0         0         0          0
 0        0      -3.07887   0         0         0          0
 0        0       0        -1.88008   0         0          0
 0        0       0         0        -1.02082   0          0
 0        0       0         0         0        -0.803554   0
 0        0       0         0         0         0         -0.618626

### Rectangular 9 x 7

In [43]:
A = randn(9, 7)

9×7 Matrix{Float64}:
 -1.09221      0.694222     …  -1.28527    -0.244773    -0.735546
 -0.142369    -0.201515        -0.343621    0.403834    -0.399011
  0.0636789   -1.14921         -0.664973    2.7235      -0.0103031
  0.486804    -0.000142874     -0.667915    0.259526    -0.122022
 -1.42227      1.30651         -0.195333   -1.05153      0.312227
  0.356179     1.45801      …  -0.670247    0.340762    -1.24939
 -2.03023     -0.6114           0.0682776   0.427084     0.490771
  0.00788358  -0.217597         2.09268     0.00153053   1.24487
 -0.351751     2.86842          0.143717   -0.307264    -0.325685

In [44]:
U, S, V, results = svd_num(A, 20, true);

In [45]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

6.799604947656927e-13

In [46]:
error_vectors = [
    abs.(results[1, 1, :]),
    abs.(results[1, 2, :]),
    abs.(results[1, 3, :]),
    abs.(results[1, 4, :]),
    abs.(results[1, 5, :]),
    abs.(results[1, 6, :])
]
labels = ["σ2", "σ3", "σ4", "σ5", "σ6", "σ7"]
plot_convergence_error(error_vectors, labels)

data: [
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, showlegend, template, title, xaxis, and yaxis"

In [47]:
# σ_2
show(stdout, "text/plain", results[2, 1, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -2.9822  3.42504  3.39679  3.44934  3.54363  3.64339  3.74057  3.84224  3.9013  3.94171  3.95035  3.95249  3.95249  3.95249  3.95249  3.95249  3.95249  3.95249  3.95249  3.95249

In [48]:
# σ_3
show(stdout, "text/plain", results[2, 2, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -2.16603  2.09585  3.32438  3.57303  3.64438  3.60867  3.5412  3.46166  3.41326  3.37998  3.37259  3.37076  3.37076  3.37076  3.37076  3.37076  3.37076  3.37076  3.37076  3.37076

In [49]:
# σ_4
show(stdout, "text/plain", results[2, 3, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 2.38026  -1.80535  2.97548  3.0506  2.9154  2.86257  2.84095  2.82918  2.82582  2.82437  2.82437  2.82437  2.82437  2.82437  2.82437  2.82437  2.82437  2.82437  2.82437  2.82437

In [50]:
# σ_5
show(stdout, "text/plain", results[2, 4, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 -2.2456  2.33447  -0.960015  1.30312  1.71047  1.91566  1.91557  1.91555  1.91555  1.91555  1.91555  1.91555  1.91555  1.91555  1.91555  1.91555  1.91555  1.91555  1.91555  1.91555

In [51]:
# σ_6
show(stdout, "text/plain", results[2, 5, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 2.08558  1.44836  1.27504  -0.825504  -0.626047  -0.55883  -0.55883  -0.55883  -0.55883  -0.55883  -0.55883  -0.55883  -0.55883  -0.55883  -0.55883  -0.55883  -0.55883  -0.55883  -0.55883  -0.55883

In [52]:
# σ_7
show(stdout, "text/plain", results[2, 6, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 1.14771  -1.17956  -1.15856  -1.15944  -1.15944  -1.15944  -1.15944  -1.15944  -1.15944  -1.15944  -1.15944  -1.15944  -1.15944  -1.15944  -1.15944  -1.15944  -1.15944  -1.15944  -1.15944  -1.15944

In [53]:
S

9×7 Matrix{Real}:
 -4.95791  0        0        0        0         0         0
  0        3.95249  0        0        0         0         0
  0        0        3.37076  0        0         0         0
  0        0        0        2.82437  0         0         0
  0        0        0        0        1.91555   0         0
  0        0        0        0        0        -0.55883   0
  0        0        0        0        0         0        -1.15944
  0        0        0        0        0         0         0
  0        0        0        0        0         0         0

### Rectangular 7 x 9

In [54]:
A = randn(7, 9)

7×9 Matrix{Float64}:
 -0.120754   1.5751    -0.293725  …  -1.20037   0.263962   0.248087
  0.629764   1.27637    1.32432      -0.721777  1.75841    1.89488
 -1.29174    0.884085  -0.351374     -0.50157   0.273119  -0.564896
  0.496827   1.46717   -0.552827     -0.750437  0.489012   0.64482
  0.157614   1.10978    0.670846     -0.510991  0.864256  -1.65031
 -0.1343    -1.55826    1.22527   …  -1.32555   0.213119   1.47736
  0.306621  -1.24583   -1.09713      -0.412868  0.709298   0.142262

In [55]:
U, S, V, results = svd_num(A, 20, true);

In [56]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

3.0737081648161095e-13

In [57]:
error_vectors = [
    abs.(results[1, 1, :]),
    abs.(results[1, 2, :]),
    abs.(results[1, 3, :]),
    abs.(results[1, 4, :]),
    abs.(results[1, 5, :]),
    abs.(results[1, 6, :])
]
labels = ["σ2", "σ3", "σ4", "σ5", "σ6", "σ7"]
plot_convergence_error(error_vectors, labels)

data: [
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, showlegend, template, title, xaxis, and yaxis"

In [58]:
# σ_2
show(stdout, "text/plain", results[2, 1, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 2.93892  3.69378  4.18834  4.15229  4.04823  3.97517  3.93929  3.92234  3.91358  3.91049  3.90917  3.90892  3.90886  3.90886  3.90886  3.90886  3.90886  3.90886  3.90886  3.90886

In [59]:
# σ_3
show(stdout, "text/plain", results[2, 2, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 3.56235  2.96812  3.2793  3.21578  3.18563  3.17564  3.17262  3.17155  3.17108  3.17098  3.17096  3.17096  3.17096  3.17096  3.17096  3.17096  3.17096  3.17096  3.17096  3.17096

In [60]:
# σ_4
show(stdout, "text/plain", results[2, 3, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 3.1399  2.24769  2.17263  2.35817  2.38329  2.388  2.38903  2.38901  2.38901  2.38901  2.38901  2.38901  2.38901  2.38901  2.38901  2.38901  2.38901  2.38901  2.38901  2.38901

In [61]:
# σ_5
show(stdout, "text/plain", results[2, 4, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 2.39148  2.20321  1.7014  1.53418  1.51444  1.51101  1.51031  1.51031  1.51031  1.51031  1.51031  1.51031  1.51031  1.51031  1.51031  1.51031  1.51031  1.51031  1.51031  1.51031

In [62]:
# σ_6
show(stdout, "text/plain", results[2, 5, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 2.05371  1.79348  1.811  1.81103  1.81103  1.81103  1.81103  1.81103  1.81103  1.81103  1.81103  1.81103  1.81103  1.81103  1.81103  1.81103  1.81103  1.81103  1.81103  1.81103

In [63]:
# σ_7
show(stdout, "text/plain", results[2, 6, :]')

1×20 adjoint(::Vector{Float64}) with eltype Float64:
 1.78631  1.78019  1.76316  1.76316  1.76316  1.76316  1.76316  1.76316  1.76316  1.76316  1.76316  1.76316  1.76316  1.76316  1.76316  1.76316  1.76316  1.76316  1.76316  1.76316

In [64]:
S

7×9 adjoint(::Matrix{Real}) with eltype Real:
 4.70765  0        0        0        0        0        0        0  0
 0        3.90886  0        0        0        0        0        0  0
 0        0        3.17096  0        0        0        0        0  0
 0        0        0        2.38901  0        0        0        0  0
 0        0        0        0        1.51031  0        0        0  0
 0        0        0        0        0        1.81103  0        0  0
 0        0        0        0        0        0        1.76316  0  0

### Badly conditioned square matrix

In [65]:
u = randn(7, 1)
u = u/norm(u)
A = (I - u * u') + 1e-9 * (rand(7, 7) - rand(7, 7))

7×7 Matrix{Float64}:
  0.996914   -0.0114422   0.0373471  …   0.0206598   0.0193143   0.0167524
 -0.0114422   0.957578    0.138465       0.0765968   0.0716083   0.0621098
  0.0373471   0.138465    0.548054      -0.25001    -0.233727   -0.202724
  0.0216899   0.0804159  -0.262475      -0.145197   -0.135741   -0.117735
  0.0206598   0.0765968  -0.25001        0.861699   -0.129294   -0.112144
  0.0193143   0.0716083  -0.233727   …  -0.129294    0.879126   -0.10484
  0.0167524   0.0621098  -0.202724      -0.112144   -0.10484     0.909066

In [66]:
U, S, V, results = svd_num(A, 25, true);

In [67]:
show(stdout, "text/plain", opnorm(U * S * V' - A, 1)/opnorm(A, 1))

9.459332210095875e-10

In [68]:
error_vectors = [
    abs.(results[1, 1, :]),
    abs.(results[1, 2, :]),
    abs.(results[1, 3, :]),
    abs.(results[1, 4, :]),
    abs.(results[1, 5, :]),
    abs.(results[1, 6, :])
]
labels = ["σ2", "σ3", "σ4", "σ5", "σ6", "σ7"]
plot_convergence_error(error_vectors, labels)

data: [
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y",
  "scatter with fields line, marker, mode, name, type, x, and y"
]

layout: "layout with fields margin, showlegend, template, title, xaxis, and yaxis"

In [69]:
# σ_2
show(stdout, "text/plain", results[2, 1, :]')

1×25 adjoint(::Vector{Float64}) with eltype Float64:
 -2.65704e-8  0.0172742  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0

In [70]:
# σ_3
show(stdout, "text/plain", results[2, 2, :]')

1×25 adjoint(::Vector{Float64}) with eltype Float64:
 0.0207387  1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0

In [71]:
# σ_4
show(stdout, "text/plain", results[2, 3, :]')

1×25 adjoint(::Vector{Float64}) with eltype Float64:
 1.0  -1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [72]:
# σ_5
show(stdout, "text/plain", results[2, 4, :]')

1×25 adjoint(::Vector{Float64}) with eltype Float64:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [73]:
# σ_6
show(stdout, "text/plain", results[2, 5, :]')

1×25 adjoint(::Vector{Float64}) with eltype Float64:
 -1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [74]:
# σ_7
show(stdout, "text/plain", results[2, 6, :]')

1×25 adjoint(::Vector{Float64}) with eltype Float64:
 1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0  -1.0

In [75]:
diag(S)

7-element Vector{Real}:
  5.501857106747788e-10
 -0.9999999991714622
 -1.0000000010398786
  1.0000000000806812
  0.9999999997863618
  1.0000000008527097
 -1.0000000005481164

In [76]:
U, S, V = svd(A);

In [77]:
show(stdout, "text/plain", opnorm(U * diagm(S) * V' - A, 1)/opnorm(A, 1))

2.3654345116920686e-15

In [78]:
S

7-element Vector{Float64}:
 1.0000000012566903
 1.0000000008922927
 1.0000000005089673
 1.000000000234326
 0.9999999994680454
 0.9999999991188824
 5.5018577221603e-10

## Time measurements

#### Bidiagonalize (without matrices - actually almost)

In [79]:
using BenchmarkTools

In [80]:
for size_m = 50:50:200
    A = randn(size_m, size_m)
    @btime bidiagonalize(A);
end

  729.294 μs (1084 allocations: 2.83 MiB)
  4.679 ms (2534 allocations: 21.31 MiB)
  16.408 ms (3984 allocations: 70.72 MiB)
  41.747 ms (5434 allocations: 166.32 MiB)


#### Bidiagonalize (with matrices)

In [81]:
for size_m = 50:50:200
    A = randn(size_m, size_m)
    @btime bidiagonalize(A, true);
end

  1.769 ms (1886 allocations: 6.84 MiB)
  13.629 ms (4136 allocations: 52.52 MiB)
  70.394 ms (6386 allocations: 175.27 MiB)
  170.831 ms (8636 allocations: 413.17 MiB)


#### From bidiag to diag (Golub Kahan) with matrices

In [82]:
execs = zeros(4, 1);
j = 1;
for size_m = 50:50:200
    A = randn(size_m, size_m)
    P_l, B, P_r = bidiagonalize(A, true);
    exec1 = @elapsed svd_golub_reinsh!(B, P_l, P_r, 2 * size_m);
    P_l, B, P_r = bidiagonalize(A, true);
    exec2 = @elapsed svd_golub_reinsh!(B, P_l, P_r, 2 * size_m);
    P_l, B, P_r = bidiagonalize(A, true);
    exec3 = @elapsed svd_golub_reinsh!(B, P_l, P_r, 2 * size_m);
    execs[j] = (exec1 + exec2 + exec3)/3;
    j = j + 1;
end

In [83]:
execs

4×1 Matrix{Float64}:
 0.42541913733333336
 1.384933174333333
 3.019284073666667
 6.374005129666666

In [84]:
show(stdout, "text/plain", opnorm(P_l * B * P_r' - A, 1)/opnorm(A, 1))

2.616273506208495e-14

#### Native SVD


In [85]:
execs = zeros(4, 1);
j = 1;
for size_m = 50:50:200
    A = randn(size_m, size_m)
    exec1 = @elapsed svd(A);
    exec2 = @elapsed svd(A);
    exec3 = @elapsed svd(A);
    execs[j] = (exec1 + exec2 + exec3)/3;
    j = j + 1;
end

In [86]:
execs

4×1 Matrix{Float64}:
 0.005753466
 0.003977679666666667
 0.009055973333333333
 0.017043341